In [ ]:
import os,sys
import numpy as np
import math
import cv2
import keras
import pickle

import glob
import pandas as pd
import imageio as io

from keras.models import load_model
from keras.layers import Input, Conv2D, average, maximum
import imgaug as ia
from imgaug import augmenters as iaa
import json
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def put_heatmap(heatmap, center, sigma):
    center_x, center_y = center[0],  center[1]
    height, width = heatmap.shape

    # modify the threshold for our desired dataset
    th = 2

    delta = math.sqrt(th * 2)

    x0 = int(max(0, center_x - delta * sigma))
    y0 = int(max(0, center_y - delta * sigma))

    x1 = int(min(width, center_x + delta * sigma))
    y1 = int(min(height, center_y + delta * sigma))

    # gaussian filter
    for y in range(y0, y1):
        for x in range(x0, x1):
            d = (x - center_x) ** 2 + (y - center_y) ** 2
            exp = d / 2.0 / sigma / sigma
            if exp > th:
                continue
            heatmap[y][x] = max(heatmap[y][x], math.exp(-exp))
            heatmap[y][x] = min(heatmap[y][x], 1.0)

    return heatmap

In [ ]:
def get_heatmap(label, target_size, scale, sigma):

    heatmap_target = []

    for coord in range(target_size[2]):
        # This is using Gaussian model but in an smart way
        heatmap = np.zeros((target_size[0], target_size[1]), dtype=np.float32)
        annos = (label[coord] * scale, label[coord + n_keypoints] * scale)
        heatmap = put_heatmap(heatmap, annos, sigma)

        heatmap_target.append(heatmap)

    #  TODO: Using Cauchuy kernel instead of Gaussian distribution

    heatmap_target = 10* np.array(heatmap_target).transpose((1, 2, 0))

    # background
    # heatmap[:, :, -1] = np.clip(1 - np.amax(heatmap, axis=2), 0.0, 1.0)

    return heatmap_target.astype(np.float32)

## Reading İzmir Data

In [ ]:
# Read all images path
path_of_images_files_list = glob.glob("C:\\Users\\Mesut\\Desktop\\5p_3p_Datas\\All_Labelling_Outputs_Miko_Mert_Cleared\\*\\*.jpg")
len(path_of_images_files_list)

In [ ]:
# Obtain the backboard images path
path_of_bb_images_list = [item for item in path_of_images_files_list if ("BBOARD" in item.split("\\")[-1] or "bboard" in item.split("\\")[-1])]
len(path_of_bb_images_list)

In [ ]:
# Read all txt path
path_of_txt_files = glob.glob("C:\\Users\\Mesut\\Desktop\\5p_3p_Datas\\All_Labelling_Outputs_Miko_Mert_Cleared\\*\\*.txt")
len(path_of_txt_files)

In [ ]:
# Obtain the backboard txt path
path_of_bb_txt_files_list = [item for item in path_of_txt_files if "BBOARD_LABELS" in item.split("\\")[-1]]
len(path_of_bb_txt_files_list)

In [ ]:
# Read Keypoints, concat it with images path and images name
image_file_name_and_keypoints_and_paths = []
for i,item in enumerate(path_of_bb_txt_files_list):
    # Pick keypoints
    keypoints = open(item, "r").read().split("\n")[1].split(";")[1:9]
    # pick backboard image path
    txt_file_name = item.split("\\")[-1]
    mutual_name = txt_file_name.split(".txt")[0].split("BBOARD_LABELS_")[-1] # Obtain the mutual_name
    bb_image_path_list = [element for element in path_of_bb_images_list if mutual_name in element]
    bb_image_path = bb_image_path_list[0]
    # Pick backboard image file name
    image_file_name = bb_image_path.split("\\")[-1]
    
    image_file_name_and_keypoints_and_paths.append([image_file_name, keypoints, bb_image_path, item])

In [ ]:
# Create our supporter Data Frame
columns = ["Image File Names", "Keypoints", "Image Paths", "Txt Paths"]
df_labels = pd.DataFrame(image_file_name_and_keypoints_and_paths, columns = columns)
df_labels

In [ ]:
def rgb2grey(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [ ]:
# Read all the backboad images
backboard_images_list = []
for path in df_labels["Image Paths"]:
    image = io.imread(path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image = cv2.imread(path, 0)
    grey = rgb2grey(image).reshape(224,224,1)
    backboard_images_list.append(grey)
backboard_images_list = np.array(backboard_images_list)
len(backboard_images_list)

## Preparing training data

In [ ]:
# Convert images list into a numpy array. This is fascinating Man! Powerful syntax!
allArrays = np.concatenate([item for item in backboard_images_list])
backboards_tensor = allArrays.reshape((len(backboard_images_list), 224,224,1))

In [ ]:
X_new = backboards_tensor
y_new = np.vstack(df_labels["Keypoints"])

In [ ]:
X_new = backboards_tensor.astype(np.float32)
y_new = np.vstack(df_labels["Keypoints"]).astype(np.float32)

In [ ]:
for i,item in enumerate(y_new):
    y_new[i] = np.hstack((item.reshape((4,2))[:,0], item.reshape((4,2))[:,1]))

In [ ]:
def visualizing_train_labels(image_tensor, y_labels, nrows=5, ncols=5):
    selection = np.random.choice(np.arange(image_tensor.shape[0]), size=(nrows*ncols), replace=False)
    fig, axes = plt.subplots(figsize=(20, 20), nrows=nrows, ncols=ncols)
    for ind, ax in zip(selection, axes.ravel()):
        img = image_tensor[ind:ind+1, :, :, :] # input shape must be (1,256,256,3)!
        #ax.imshow(img[0]) # for plotting input shape must be: (256,256,3)
        ax.imshow(img[0].reshape(224,224), cmap=plt.get_cmap("gray")) # for plotting input shape must be: (256,256,3)
        keypoints = y_labels[ind]
        ax.plot(keypoints.reshape((2,4))[0,:],keypoints.reshape((2,4))[1,:], 'ro')
        #ax.plot(np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[0,0:4], np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[1,0:4], 'ro')
        ax.set_title(str(ind))
        ax.axis('off')

In [ ]:
visualizing_train_labels(X_new, y_new)

## Try Heatmap

In [ ]:
n_keypoints = 4
a = get_heatmap(y_new[0], (224,224,4), 1, 2)

In [ ]:
a.shape

In [ ]:
plt.imshow(a[:,:,2], cmap=plt.get_cmap("gray"))

In [ ]:
plt.imshow(X_new[0].reshape(224,224), cmap=plt.get_cmap("gray"))

In [ ]:
a[:,:,2].shape

In [ ]:

plt.imshow(np.sum(a, axis = -1), cmap=plt.get_cmap("gray"))

In [ ]:
def visualizing_masks(image_tensor, y_labels, nrows=5, ncols=5):
    selection = np.random.choice(np.arange(image_tensor.shape[0]), size=(nrows*ncols), replace=False)
    fig, axes = plt.subplots(figsize=(20, 20), nrows=nrows, ncols=ncols)
    for ind, ax in zip(selection, axes.ravel()):
        img = image_tensor[ind:ind+1, :, :, :] # input shape must be (1,256,256,3)!
        #ax.imshow(img[0]) # for plotting input shape must be: (256,256,3)
        keypoints = y_labels[ind]
        a = get_heatmap(keypoints, (224,224,4), 1, 5)
        summed = np.sum(a, axis = -1)
        #summed = img[0].reshape(224,224) + np.sum(a, axis = -1)
        max_of_summed = summed.max()
        ax.imshow(img[0].reshape(224,224) + summed*(255/max_of_summed), cmap=plt.get_cmap("gray"))
        #axes[0,c].imshow(summed, cmap=plt.get_cmap("gray")) # for plotting input shape must be: (256,256,3)
        #axes[1,c].imshow(np.sum(a, axis = -1), cmap=plt.get_cmap("gray"))
        #ax.plot(keypoints.reshape((2,4))[0,:],keypoints.reshape((2,4))[1,:], 'ro')
        #ax.plot(np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[0,0:4], np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[1,0:4], 'ro')
        ax.set_title(str(ind))
        #axes[1,c].set_title(str(ind))
        ax.axis('off')
        #axes[1,c].axis('off')

In [ ]:
X_new[0].reshape(224,224).shape

In [ ]:
all_a = np.sum(a, axis = -1)
max1 = all_a.max()
print(max1)
plt.imshow(X_new[0].reshape(224,224) + all_a *(255/max1), cmap=plt.get_cmap("gray"))


In [ ]:
visualizing_masks(X_new, y_new, nrows=5, ncols=5)

## Generate Heatmap Tensor

In [ ]:
n_keypoints = 4
a = get_heatmap(y_new[0], (224,224,4), 1, 5)

In [ ]:
n_keypoints = 4
heatmap_list = []
for item in y_new:
    heatmap = get_heatmap(item, (224,224,4), 1, 2)
    heatmap_list.append(heatmap)
len(heatmap_list)

In [ ]:
heatmap_list[0].shape

In [ ]:
all_heatmap_array = np.vstack(heatmap_list)
y_heatmap_tensor = all_heatmap_array.reshape(len(heatmap_list), 224,224,4)
y_heatmap_tensor.shape

## Apply Train Test Split for Heatmap

In [ ]:
test_size = 0.2
test_selection_new = np.random.choice(np.arange(X_new.shape[0]), size=int(X_new.shape[0]*test_size), replace=False)
len(test_selection_new)

In [ ]:
X_new_train = np.array([image for index,image in enumerate(X_new) if index not in test_selection_new])
X_new_train.shape

In [ ]:
y_heatmap_train = np.array([keypoints for index,keypoints in enumerate(y_heatmap_tensor) if index not in test_selection_new])
y_heatmap_train.shape

In [ ]:
# Normalizing the inputs
X_new_train = X_new_train / 255.
#y_heatmap_train = y_heatmap_train / 224.

In [ ]:
def visualizing_train_heatmaps(image_tensor, y_labels, nrows=5, ncols=5):
    selection = np.random.choice(np.arange(image_tensor.shape[0]), size=(nrows*ncols), replace=False)
    fig, axes = plt.subplots(figsize=(20, 20), nrows=nrows, ncols=ncols)
    for ind, ax in zip(selection, axes.ravel()):
        img = image_tensor[ind:ind+1, :, :, :] # input shape must be (1,256,256,3)!
        #ax.imshow(img[0]) # for plotting input shape must be: (256,256,3)
        #ax.imshow(img[0].reshape(224,224), cmap=plt.get_cmap("gray")) # for plotting input shape must be: (256,256,3)
        summed = np.sum(y_labels[ind], axis = -1)
        max_of_summed = summed.max()
        ax.imshow((img[0].reshape(224,224)*255) + (summed*(255/max_of_summed)), cmap=plt.get_cmap("gray"))
        
        #keypoints = y_labels[ind]
        #ax.plot(keypoints.reshape((2,4))[0,:]*224,keypoints.reshape((2,4))[1,:]*224, 'ro')
        #ax.plot(np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[0,0:4], np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[1,0:4], 'ro')
        ax.set_title(str(ind))
        ax.axis('off')

In [ ]:
ind = 0
img = X_new_train[ind:(ind+1),:,:,:]
summed = np.sum(y_heatmap_train[ind], axis = -1)
max_of_summed = summed.max()
plt.imshow((img[0].reshape(224,224)*255) + summed*(255/max_of_summed), cmap=plt.get_cmap("gray"))
#plt.imshow(summed*(255/max_of_summed)*224, cmap=plt.get_cmap("gray"))
#plt.imshow(img[0].reshape(224,224)*255, cmap=plt.get_cmap("gray"))

In [ ]:
visualizing_train_heatmaps(X_new_train, y_heatmap_train)

## Training Part(Oh!)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.layers import Reshape
from keras.models import load_model

In [ ]:
import keras.backend as K
import keras.utils

from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Activation, Add, Conv2D, Conv2DTranspose, concatenate, Cropping2D, MaxPooling2D, Reshape, UpSampling2D
from keras.models import Input, Model
from keras.optimizers import SGD, RMSprop, Adam
from keras.regularizers import l2

In [ ]:
def UNET(input_shape):
    def downsample_block(x, block_num, n_filters, pooling_on=True):

        x = Conv2D(n_filters, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
                   name="Block" + str(block_num) + "_Conv1")(x)
        x = Conv2D(n_filters, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
                   name="Block" + str(block_num) + "_Conv2")(x)
        skip = x

        if pooling_on is True:
            x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', name="Block" + str(block_num) + "_Pool1")(x)

        return x, skip

    def upsample_block(x, skip, block_num, n_filters):

        x = Conv2DTranspose(n_filters, kernel_size=(2, 2), strides=2, padding='valid', activation='relu',
                            name="Block" + str(block_num) + "_ConvT1")(x)
        x = concatenate([x, skip], axis=-1, name="Block" + str(block_num) + "_Concat1")
        x = Conv2D(n_filters, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
                   name="Block" + str(block_num) + "_Conv1")(x)
        x = Conv2D(n_filters, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
                   name="Block" + str(block_num) + "_Conv2")(x)

        return x

    input = Input(input_shape, name="Input")

    # downsampling
    x, skip1 = downsample_block(input, 1, 64)
    x, skip2 = downsample_block(x, 2, 128)
    x, skip3 = downsample_block(x, 3, 256)
    x, skip4 = downsample_block(x, 4, 512)
    x, _ = downsample_block(x, 5, 1024, pooling_on=False)

    # upsampling
    x = upsample_block(x, skip4, 6, 512)
    x = upsample_block(x, skip3, 7, 256)
    x = upsample_block(x, skip2, 8, 128)
    x = upsample_block(x, skip1, 9, 64)

    output = Conv2D(15, kernel_size=(1, 1), strides=1, padding='valid', activation='linear', name="output")(x)
    output = Reshape(target_shape=(96*96*15,1))(output)

    model = Model(inputs=input, outputs=output, name="Output")

    return model


unet = UNET(input_shape=(96, 96, 1))
print(unet.summary())

In [ ]:
# building network here
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size), \
               kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size), \
               kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)

    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)

    outputs = Conv2D(4, (1, 1), activation='sigmoid')(c9)
    output = Reshape(target_shape=(224*224*4, 1))(outputs)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model


In [ ]:
inputimg = Input((224,224,1))
model = get_unet(inputimg, n_filters = 16, dropout = 0.1, batchnorm = True)

In [ ]:
model.summary()

In [ ]:
# Define L2 norm
def customLoss_L2(y_true, y_pred):
    channel_loss = K.sum(K.square(y_pred - y_true), axis=-1)
    total_loss = K.mean(channel_loss, axis=-1)
    print(total_loss.shape)
    return total_loss

In [ ]:
# Define L1 norm
def customLoss_L1(y_true, y_pred):
    channel_loss = K.sum(K.abs(y_pred - y_true), axis=-1)
    total_loss = K.mean(channel_loss, axis=-1)
    print(total_loss.shape)
    return total_loss

In [ ]:
# Define w1*L2 + w2*L1 norm(regularized)
def customLoss_L2L1(y_true, y_pred, w1 = 1, w2 = 10):
    channel_loss_l2 = K.sum(K.square(y_pred - y_true), axis=-1)
    total_loss_l2 = K.mean(channel_loss_l2, axis=-1)
    
    channel_loss_l1 = K.sum(K.abs(y_pred - y_true), axis=-1)
    total_loss_l1 = K.mean(channel_loss_l1, axis=-1)
    
    regularized_loss = w1*total_loss_l2 + w2*total_loss_l1
    
    return regularized_loss

## Case 1: L2 Norm

In [ ]:
model.compile(optimizer="adam", loss=customLoss_L2, metrics=['mse'])

In [ ]:
epochs = 3
history_heatmap = model.fit(X_new_train, y_heatmap_train, 
                 validation_split=0.08, shuffle=True, 
                 epochs=epochs, batch_size=32)

In [ ]:
def visualizing_predictions(model_heat, X_data, test_select, nrows=5, ncols=5):
    selection = np.random.choice(test_select, size=(nrows*ncols), replace=False)
    fig, axes = plt.subplots(figsize=(20, 20), nrows=nrows, ncols=ncols)
    for ind, ax in zip(selection, axes.ravel()):
        img = X_data[ind:ind+1, :, :, :]/255 # input shape must be (1,256,256,3)!
        predictions = model_heat.predict(img)*224
        #ax.imshow(img[0]) # for plotting input shape must be: (256,256,3)
        #ax.imshow(img[0].reshape(224,224), cmap=plt.get_cmap("gray")) # for plotting input shape must be: (256,256,3)
        summed = np.sum(predictions, axis = -1)
        max_of_summed = summed.max()
        ax.imshow((img[0].reshape(224,224)*255) + (summed.reshape((224,224))*(255/max_of_summed)), cmap=plt.get_cmap("gray"))
        
        #keypoints = y_labels[ind]
        #ax.plot(keypoints.reshape((2,4))[0,:]*224,keypoints.reshape((2,4))[1,:]*224, 'ro')
        #ax.plot(np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[0,0:4], np.array(df_label.iloc[ind]["Keypoints"]).reshape((2,5))[1,0:4], 'ro')
        ax.set_title(str(ind))
        ax.axis('off')

In [ ]:
plt.imshow(np.sum(model.predict(X_new[0:0+1, :, :, :]/255)*224, axis = -1).reshape((224,224)), cmap=plt.get_cmap("gray")) 

In [ ]:
np.sum(model.predict(X_new[0:0+1, :, :, :]/255)*224, axis = -1).max()

In [ ]:
np.sum(model.predict(X_new[0:0+1, :, :, :]/255)*224, axis = -1).reshape((224,224)).max()

In [ ]:
np.sum(model.predict(X_new[0:0+1, :, :, :]/255)*224, axis = -1).reshape((224,224))

In [ ]:
plt.imshow(np.sum((model.predict(X_new[0:0+1, :, :, :]/255)*224).reshape((224,224,4)), axis = -1), cmap = plt.get_cmap("gray"))

In [ ]:
np.sum(model.predict(X_new[0:0+1, :, :, :]/255), axis = -1).reshape((224,224))

In [ ]:
model

In [ ]:
X_new[0].shape

In [ ]:
visualizing_predictions(model, X_new, test_selection_new)

In [ ]:
def plot_predictions(model, X_data, test_select, nrows=5, ncols=5):
    """Plots sampled faces with their truth and predictions."""
    selection = np.random.choice(test_select, size=(nrows*ncols), replace=False)
    fig, axes = plt.subplots(figsize=(15, 15), nrows=nrows, ncols=ncols)
    for ind, ax in zip(selection, axes.ravel()):
        img = X_data[ind:ind+1, :, :, :] / 255. # input shape must be (1,256,256,3)!
        predictions = model.predict(img)*224
        ax.imshow(img[0].reshape(224,224), cmap=plt.get_cmap("gray")) # for plotting input shape must be: (256,256,3)
        #ax.imshow(img[0]) # for plotting input shape must be: (256,256,3)
        ax.plot(predictions.reshape((2,4))[0,:],predictions.reshape((2,4))[1,:], 'bo')
        ax.set_title(str(ind))
        ax.axis('off')
    fig.suptitle("Model X", fontsize = 20) 

## Case 2: L1 norm

In [ ]:
inputimg = Input((224,224,1))
model_l1 = get_unet(inputimg, n_filters = 16, dropout = 0.1, batchnorm = True)

In [ ]:
model_l1.compile(optimizer="adam", loss=customLoss_L1, metrics=['mse'])

In [ ]:
epochs = 3
history_heatmap = model_l1.fit(X_new_train, y_heatmap_train, 
                 validation_split=0.08, shuffle=True, 
                 epochs=epochs, batch_size=32)

In [ ]:
plt.imshow(np.sum((model_l1.predict(X_new[0:0+1, :, :, :]/255)*224).reshape((224,224,4)), axis = -1), cmap = plt.get_cmap("gray"))

In [ ]:
visualizing_predictions(model_l1, X_new, test_selection_new)

## Case 3: L2 + L1(weighted)

In [ ]:
inputimg = Input((224,224,1))
model_l2l1 = get_unet(inputimg, n_filters = 16, dropout = 0.1, batchnorm = True)

In [ ]:
model_l2l1.compile(optimizer="adam", loss=customLoss_L2L1, metrics=['mse'])

In [ ]:
epochs = 3
history_heatmap = model_l2l1.fit(X_new_train, y_heatmap_train, 
                 validation_split=0.08, shuffle=True, 
                 epochs=epochs, batch_size=32)

In [ ]:
visualizing_predictions(model_l2l1, X_new, test_selection_new)